In [1]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup
import json
import talib
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
def get_datadf(symbol):
    raw_data = requests.get("https://query1.finance.yahoo.com/v8/finance/chart/"+ symbol +"?period1=1349893800&period2=1507660200&interval=1d").text
    raw_data = json.loads(raw_data)
    df = raw_data[u'chart'][u'result'][0][u'indicators'][u'quote'][0]
    df['unadjclose'] = raw_data[u'chart'][u'result'][0][u'indicators'][u'unadjclose'][0][u'unadjclose']
    df['adjclose'] = raw_data[u'chart'][u'result'][0][u'indicators'][u'adjclose'][0][u'adjclose']
    df['timestamp'] = raw_data[u'chart'][u'result'][0][u'timestamp']
    data = pd.DataFrame.from_dict(df)
    df['sym'] = 'INFY'
    data.index = pd.to_datetime(data['timestamp'], unit='s').values
    data = data.dropna()
    data.drop(['unadjclose','timestamp'],axis=1, inplace=True)
    return data

In [4]:
def returns(arr):
    arr_trans = [0] * 5
    arr_trans[0] = [0] * len(arr.columns)
    arr_trans[1:] = (arr.iloc[1:,].values - arr.iloc[:-1,].values) / arr.iloc[:-1].values
    arr_trans[0] = np.array([0.] * len(arr.columns))
    return arr_trans

def standardize(arr):
    return (arr - arr.mean()) / arr.std()

## Indicators

In [5]:
def getIndicators(df,macdfast=12, macdslow=26, signal=9, rsitime=9):
    indicators = pd.DataFrame()
    macd12, macdsignal12, macdhist12 = talib.MACD(df.close.values, fastperiod=macdfast, slowperiod=macdslow, signalperiod=signal)
    indicators.loc[:,'macd'] = macd12
    indicators.loc[:,'macdhist'] = macdhist12 
    
    indicators.loc[:,'ema10'] = talib.EMA(df.close.values, timeperiod=10)
    indicators.loc[:,'ema20'] = talib.EMA(df.close.values, timeperiod=20)
    indicators.loc[:,'ema30'] = talib.EMA(df.close.values, timeperiod=30)
    indicators.loc[:,'ema40'] = talib.EMA(df.close.values, timeperiod=40)
    indicators.loc[:,'ema50'] = talib.EMA(df.close.values, timeperiod=50)
    
    indicators.loc[:,'arrondown7'], indicators.loc[:,'arronup7'] = talib.AROON(df.high.values, df.low.values, timeperiod=7)
    indicators.loc[:,'arrondown13'], indicators.loc[:,'arronup13'] = talib.AROON(df.high.values, df.low.values, timeperiod=13)
    
    upperband, middleband, lowerband = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    indicators.loc[:,'bbindic'] = (df.close.values - lowerband) / (upperband - lowerband)
    
    rsi = talib.RSI(df.close.values, timeperiod=rsitime)
    indicators.loc[:,'rsi'] = rsi
    indicators.index = df.index
    return indicators

## Patterns

In [6]:
def getPatterns(df):
    patterns = pd.DataFrame()
    patterns['marubuzo'] = talib.CDLXSIDEGAP3METHODS(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['spinningtop'] = talib.CDLSPINNINGTOP(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['doji'] = talib.CDLDOJI(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['hammer'] = talib.CDLHAMMER(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['hangingman'] = talib.CDLHANGINGMAN(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['shootigstar'] = talib.CDLSHOOTINGSTAR(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['engulfing'] = talib.CDLENGULFING(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['piercing'] = talib.CDLPIERCING(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['darkcloudcover'] = talib.CDLDARKCLOUDCOVER(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['harami'] = talib.CDLHARAMI(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['gapthreemethod'] = talib.CDLXSIDEGAP3METHODS(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['morningstar'] = talib.CDLMORNINGSTAR(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns['eveningstar'] = talib.CDLEVENINGSTAR(df.open.values, df.high.values, df.low.values, df.close.values)
    patterns.index = df.index
    patterns = patterns / 100
    return patterns

In [7]:
def merge_all(data):
    indicators = getIndicators(data)
    indicators = indicators.bfill()

    patterns = getPatterns(data)

    sym_data = pd.merge(indicators, patterns, left_index=True, right_index=True)
    final_data =  pd.merge(sym_data, data, left_index=True, right_index=True)
    return final_data

In [8]:
def transformations(data):
    data['adjclose'][1:] = (data['adjclose'][1:,].values - data['adjclose'][:-1,].values) / data['adjclose'][:-1].values
    data['adjclose'][0] = 0
    data.loc[data.adjclose < 0,'volume'] = data.loc[data.adjclose < 0,'volume'] * -1
    data['signal'] = data['adjclose'].apply(lambda x:1 if x >= 0.03 else 0)
    data.signal = np.append(data.signal[1:].values,0)

    data['high'] = (data['high'] - data['close']) /  data['close']
    data['low'] = (data['close'] - data['low']) /  data['close']
    data['open'] = (data['close'] - data['open']) /  data['close']
    data['volume'] = standardize(data.volume)
    
    data.loc[:,['macd','ema10','ema20','ema30','ema40','ema50']] = returns(data.loc[:,['macd','ema10','ema20','ema30','ema40','ema50']])
    data.loc[:,['macdhist','arrondown7','arronup7','arrondown13', 'arronup13', 'rsi']] = standardize(data.loc[:,['macdhist','arrondown7','arronup7','arrondown13', 'arronup13', 'rsi']])
    data.dropna(inplace=True)
    return data

In [62]:
def accuracy(y_true, y_pred):
    from sklearn.metrics import precision_recall_fscore_support
    print(precision_recall_fscore_support(testy, test_predy, average='macro'))
    print(confusion_matrix(testy, test_predy))

In [13]:
nifty_all = ['ADANIPORTS.NS', 'AMBUJACEM.NS', 'ASIANPAINT.NS', 'AUROPHARMA.NS', 'AXISBANK.NS', \
             'BAJFINANCE.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'INFRATEL.NS', 'BOSCHLTD.NS', 'CIPLA.NS', 'COALINDIA.NS', \
             'DRREDDY.NS', 'EICHERMOT.NS', 'GAIL.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS',\
             'HINDPETRO.NS', 'HINDUNILVR.NS', 'HDFCBANK.NS', 'ITC.NS', 'ICICIBANK.NS', 'IBREALEST.NS', 'IOC.NS',\
              'INFY.NS', 'KOTAKBANK.NS', 'LT.NS', 'LUPIN.NS', 'M&M.NS', 'MARUTI.NS', 'NTPC.NS', \
             'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', \
             'TECHM.NS', 'UPL.NS', 'ULTRACEMCO.NS', 'VEDL.NS', 'WIPRO.NS', 'YESBANK.NS', 'ZEEL.NS'] #'INDUSINDBK.NS',

In [10]:
stocks = pd.DataFrame()

In [14]:
for stock in nifty_all:
    print(stock)
    data = get_datadf(stock)
    merge_data = merge_all(data)
    stocks = stocks.append(transformations(merge_data))

ADANIPORTS.NS
AMBUJACEM.NS
ASIANPAINT.NS
AUROPHARMA.NS
AXISBANK.NS
BAJFINANCE.NS
BPCL.NS
BHARTIARTL.NS
INFRATEL.NS
BOSCHLTD.NS
CIPLA.NS
COALINDIA.NS
DRREDDY.NS
EICHERMOT.NS
GAIL.NS
HCLTECH.NS
HDFCBANK.NS
HEROMOTOCO.NS
HINDALCO.NS
HINDPETRO.NS
HINDUNILVR.NS
HDFCBANK.NS
ITC.NS
ICICIBANK.NS
IBREALEST.NS
IOC.NS
INFY.NS
KOTAKBANK.NS
LT.NS
LUPIN.NS
M&M.NS
MARUTI.NS
NTPC.NS
ONGC.NS
POWERGRID.NS
RELIANCE.NS
SBIN.NS
SUNPHARMA.NS
TCS.NS
TATAMOTORS.NS
TATASTEEL.NS
TECHM.NS
UPL.NS
ULTRACEMCO.NS
VEDL.NS
WIPRO.NS
YESBANK.NS
ZEEL.NS


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
y = stocks.signal
x = stocks.drop('signal', axis=1)

In [17]:
trainx, testx, trainy, testy = train_test_split(x,y, test_size=0.2)

# Feature Selection

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
clf = RandomForestClassifier(n_estimators=500, verbose=2, n_jobs=2)

In [20]:
clf.fit(trainx, trainy)

building tree 1 of 500building tree 2 of 500

building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   14.4s


building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68 of 500
building tree 69 of 500
building tree 70 of 500
building tree 71 of 500
building tree 72 of 500
building tree 73 of 500
building tree 74 of 500
building tree 75 of 500
building tree 76 of 500
building tree 77 of 500
building tree 78 of 500
building tree 79 of 500
building tree 80

[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:  1.1min


building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500
building tree 165 of 500
building tree 166 of 500
building tree 167 of 500
building tree 168 of 500
building tree 169 of 500
building tree 170 of 500
building tree 171 of 500
building tree 172 of 500
building tree 173 of 500
building tree 174 of 500
building tree 175 of 500
building tree 176 of 500
building tree 177 of 500
building tree 178 of 500
building tree 179 of 500
building tree 180 of 500
building tree 181 of 500
building tree 182 of 500
building tree 183 of 500
building tree 184 of 500
building tree 185 of 500
building tree 186 of 500
building tree 187 of 500
building tree 188 of 500
building tree 189 of 500
building tree 190 of 500
building tree 191 of 500
building tree 192 of 500
building tree 193 of 500
building tree 194 of 500
building tree 195 of 500
building tree 196 of 500
building tree 197 of 500
building tree 198 of 500
building tree 199 of 500


[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:  2.4min


building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500
building tree 373 of 500
building tree 374 of 500
building tree 375 of 500
building tree 376 of 500
building tree 377 of 500
building tree 378 of 500
building tree 379 of 500
building tree 380 of 500
building tree 381 of 500
building tree 382 of 500
building tree 383 of 500
building tree 384 of 500
building tree 385 of 500
building tree 386 of 500
building tree 387 of 500
building tree 388 of 500
building tree 389 of 500
building tree 390 of 500
building tree 391 of 500
building tree 392 of 500
building tree 393 of 500
building tree 394 of 500
building tree 395 of 500
building tree 396 of 500
building tree 397 of 500
building tree 398 of 500
building tree 399 of 500
building tree 400 of 500
building tree 401 of 500
building tree 402 of 500


[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:  3.3min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=2, oob_score=False, random_state=None,
            verbose=2, warm_start=False)

In [21]:
important_feature = clf.feature_importances_

In [22]:
indices = np.argsort(clf.feature_importances_[::-1])

In [23]:
features_rf = pd.DataFrame(important_feature, index=trainx.columns, columns=['features'])

In [24]:
columns = features_rf.sort_values(by='features', ascending=False)[:19].index

# Feature Scaling

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
ss = StandardScaler()

In [27]:
trainx_scale = ss.fit_transform(trainx.loc[:,columns])
testx_scale = ss.fit_transform(testx.loc[:,columns])

# Logistic Regression

In [77]:
from sklearn.linear_model import LogisticRegression

In [118]:
clf_lr = LogisticRegression(penalty='l2',C=1000.0)

In [119]:
clf_lr.fit(trainx_scale, trainy)

LogisticRegression(C=1000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [120]:
test_predy = clf_lr.predict(testx_scale)

In [37]:
from sklearn.metrics import confusion_matrix

In [121]:
confusion_matrix(testy, test_predy)

array([[17202,    14],
       [ 1154,    11]], dtype=int64)

# Gradient Boosting

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [49]:
clf_gb = GradientBoostingClassifier(n_estimators=600,learning_rate=0.5, verbose=2)

In [50]:
clf_gb.fit(trainx_scale , trainy)

      Iter       Train Loss   Remaining Time 
         1           0.4652            1.95m
         2           0.4592            1.97m
         3           0.4561            1.96m
         4           0.4531            1.98m
         5           0.4508            1.96m
         6           0.4488            1.96m
         7           0.4477            1.94m
         8           0.4464            1.94m
         9           0.4450            1.93m
        10           0.4445            1.90m
        11           0.4432            1.89m
        12           0.4419            1.88m
        13           0.4410            1.87m
        14           0.4393            1.87m
        15           0.4387            1.85m
        16           0.4382            1.85m
        17           0.4369            1.85m
        18           0.4358            1.86m
        19           0.4350            1.85m
        20           0.4348            1.84m
        21           0.4344            1.82m
        2

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.5, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=600, presort='auto', random_state=None,
              subsample=1.0, verbose=2, warm_start=False)

In [51]:
test_predy = clf_gb.predict(testx_scale)

In [46]:
test_pred_prb = clf_gb.predict_proba(testx_scale)

In [61]:
accuracy(testy, test_predy)

(0.6010916528797936, 0.54757480283922777, 0.56077200761660384, None)
[[22384   530]
 [ 1394   187]]


# XGBoost

In [65]:
import xgboost as xgb

In [94]:
gbm = xgb.XGBClassifier(max_depth=7, n_estimators=1500, learning_rate=0.6).fit(trainx_scale , trainy)

In [95]:
test_predy = gbm.predict(testx_scale)

In [96]:
accuracy(testy, test_predy)

(0.91823873800617983, 0.65447381091148671, 0.71805792996139994, None)
[[22848    66]
 [ 1088   493]]


In [93]:
pd.Series(test_predy).value_counts()

0    23948
1      547
dtype: int64